## Skin Cancer Detection 2 - bottleneck features

In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras import applications
from sklearn.datasets import load_files
from keras.utils import np_utils
# from glob import glob

# dimensions of our images.
img_width, img_height = 254, 254

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/valid'
nb_train_samples = 2000
nb_validation_samples = 150
epochs =100
batch_size = 15

def load_dataset(path, categories_count):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), categories_count)
    return files, targets

def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG19(
        include_top=False, 
        weights='imagenet')

#     generator = datagen.flow_from_directory(
#         train_data_dir,
#         target_size=(img_width, img_height),
#         batch_size=batch_size,
#         class_mode="categorical",
#         shuffle=False)
    
#     bottleneck_features_train = model.predict_generator(
#         generator, nb_train_samples // batch_size)
    
#     np.save(open('bottleneck_features_train2.npy', 'wb'),
#             bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    
    print(nb_validation_samples // batch_size)
    
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    
    np.save(open('bottleneck_features_validation2.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train2.npy', 'rb'))
    train_labels = load_dataset(train_data_dir, 3)[1]

    validation_data = np.load(open('bottleneck_features_validation2.npy', 'rb'))
    validation_labels = load_dataset(validation_data_dir, 3)[1]
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    
    
def train_top_model2():
    train_data = np.load(open('bottleneck_features_train2.npy', 'rb'))
    train_labels = load_dataset(train_data_dir, 3)[1]

    validation_data = np.load(open('bottleneck_features_validation2.npy', 'rb'))
    validation_labels = load_dataset(validation_data_dir, 3)[1]
    
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(2,2), strides=(1,1), padding='same', input_shape=(7, 7, 512), activation="relu"))
    model.add(Dropout(0.5))
    model.add(Conv2D(filters=16, kernel_size=(2,2), strides=(1,1), padding='same', activation="relu"))
    model.add(Dropout(0.5))
    model.add(Conv2D(filters=8, kernel_size=(2,2), strides=(1,1), padding='same', activation="relu"))
    model.add(Dropout(0.5))
    model.add(GlobalAveragePooling2D(data_format="channels_last"))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='sigmoid'))
    
    model.summary()

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)




Using TensorFlow backend.


In [2]:
train_top_model2()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 7, 7, 32)          65568     
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 16)          2064      
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 7, 16)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 8)           520       
_________________________________________________________________
dropout_3 (Dropout)          (None, 7, 7, 8)           0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 8)                 0         
__________

2000/2000 [==============================] - 1s 403us/step - loss: 0.8169 - acc: 0.6860 - val_loss: 1.1415 - val_acc: 0.5200
Epoch 50/100
2000/2000 [==============================] - 1s 411us/step - loss: 0.8141 - acc: 0.6860 - val_loss: 1.2226 - val_acc: 0.5200
Epoch 51/100
2000/2000 [==============================] - 1s 400us/step - loss: 0.8063 - acc: 0.6860 - val_loss: 1.1230 - val_acc: 0.5200
Epoch 52/100
2000/2000 [==============================] - 1s 405us/step - loss: 0.8133 - acc: 0.6860 - val_loss: 1.6151 - val_acc: 0.5200
Epoch 53/100
2000/2000 [==============================] - 1s 405us/step - loss: 0.8102 - acc: 0.6860 - val_loss: 1.1360 - val_acc: 0.5200
Epoch 54/100
2000/2000 [==============================] - 1s 405us/step - loss: 0.8114 - acc: 0.6860 - val_loss: 1.1268 - val_acc: 0.5200
Epoch 55/100
2000/2000 [==============================] - 1s 463us/step - loss: 0.8142 - acc: 0.6860 - val_loss: 1.3430 - val_acc: 0.5200
Epoch 56/100
2000/2000 [=======================

In [3]:
#     train_data = np.load(open('bottleneck_features_train2.npy', 'rb'))
#     train_labels = load_dataset(train_data_dir, 3)[1]

#     validation_data = np.load(open('bottleneck_features_validation2.npy', 'rb'))
#     validation_labels = load_dataset(validation_data_dir, 3)[1]
    
#     model = Sequential()
#     model.add(Flatten(input_shape=train_data.shape[1:]))

    

In [4]:
# model = Sequential()
# model.add(Conv2D(10))

In [5]:
# save_bottlebeck_features()

In [6]:
train_data = np.load(open('bottleneck_features_train2.npy', 'rb'))

In [7]:
train_data.shape

(2000, 7, 7, 512)

In [8]:
train_top_model()

Train on 2000 samples, validate on 150 samples
Epoch 1/100
2000/2000 [==============================] - 2s 852us/step - loss: 1.2203 - acc: 0.2410 - val_loss: 1.0986 - val_acc: 0.2000
Epoch 2/100
2000/2000 [==============================] - 1s 627us/step - loss: 1.1833 - acc: 0.2290 - val_loss: 1.0986 - val_acc: 0.2000
Epoch 3/100
2000/2000 [==============================] - 1s 628us/step - loss: 1.1520 - acc: 0.2110 - val_loss: 1.0986 - val_acc: 0.2000
Epoch 4/100
2000/2000 [==============================] - 1s 628us/step - loss: 1.1023 - acc: 0.1925 - val_loss: 1.0986 - val_acc: 0.2000
Epoch 5/100
2000/2000 [==============================] - 1s 630us/step - loss: 1.1204 - acc: 0.2095 - val_loss: 1.0986 - val_acc: 0.2000
Epoch 6/100
2000/2000 [==============================] - 1s 629us/step - loss: 1.1382 - acc: 0.1990 - val_loss: 1.0986 - val_acc: 0.2000
Epoch 7/100
2000/2000 [==============================] - 1s 636us/step - loss: 1.1637 - acc: 0.2025 - val_loss: 1.0970 - val_acc: 0

2000/2000 [==============================] - 1s 637us/step - loss: 0.7748 - acc: 0.6900 - val_loss: 2.2909 - val_acc: 0.5200
Epoch 61/100
2000/2000 [==============================] - 1s 639us/step - loss: 0.7921 - acc: 0.6900 - val_loss: 1.9097 - val_acc: 0.5200
Epoch 62/100
2000/2000 [==============================] - 1s 658us/step - loss: 0.7554 - acc: 0.6870 - val_loss: 1.9429 - val_acc: 0.5200
Epoch 63/100
2000/2000 [==============================] - 1s 640us/step - loss: 0.7557 - acc: 0.6895 - val_loss: 1.6883 - val_acc: 0.5200
Epoch 64/100
2000/2000 [==============================] - 1s 647us/step - loss: 0.7521 - acc: 0.6940 - val_loss: 1.7082 - val_acc: 0.5200
Epoch 65/100
2000/2000 [==============================] - 1s 634us/step - loss: 0.7733 - acc: 0.6920 - val_loss: 1.9978 - val_acc: 0.5200
Epoch 66/100
2000/2000 [==============================] - 1s 638us/step - loss: 0.7638 - acc: 0.6910 - val_loss: 1.8493 - val_acc: 0.5200
Epoch 67/100
2000/2000 [=======================